In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [81]:
train_images = pd.read_csv("/content/drive/MyDrive/Colab Data/mnist_train.csv")
test_images = pd.read_csv("/content/drive/MyDrive/Colab Data/mnist_test.csv")

In [82]:
train_labels = train_images.pop("label")
test_labels = test_images.pop("label")

In [83]:
print("First training image sample pattern")
print(f"Label: {train_labels[0]}")
ind = 0
for _, val in train_images.iloc[0].iteritems():
    if ind % 28 == 0:
        print("\n")
    if val > 0:
        print("*", end=" ")
    else:
        print(" ", end=" ")
    ind += 1

First training image sample pattern
Label: 5


                                                        

                                                        

                                                        

                                                        

                                                        

                        * * * * * * * * * * * *         

                * * * * * * * * * * * * * * * *         

              * * * * * * * * * * * * * * * *           

              * * * * * * * * * * *                     

                * * * * * * *   * *                     

                  * * * * *                             

                      * * * *                           

                      * * * *                           

                        * * * * * *                     

                          * * * * * *                   

                            * * * * * *                 

                         

In [84]:
def standardize(train, test):
    mean = np.mean(train)
    std = np.std(train)
    return ((train - mean) / std).fillna(0), ((test - mean) / std).fillna(0)

In [85]:
train_images, test_images = standardize(train_images, test_images)

In [86]:
def one_hot_labels(train, test):
    y_train = np.zeros((train.shape[0], 10))
    y_test = np.zeros((test.shape[0], 10))
    for i, y in enumerate(train):
        y_train[i][y] = 1
    for i, y in enumerate(test):
        y_test[i][y] = 1
    return pd.DataFrame(data=y_train), pd.DataFrame(data=y_test)

In [87]:
train_labels, test_labels = one_hot_labels(train_labels, test_labels)

In [88]:
X_train, X_test, y_train, y_test = train_images, test_images, train_labels, test_labels

In [89]:
X_train.head(3)

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,2x12,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004418,-0.005755,-0.004083,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004083,-0.00471,-0.008799,-0.011591,-0.014759,-0.019285,-0.024672,-0.029071,...,-0.135433,-0.11648,-0.093951,-0.072577,-0.052364,-0.036383,-0.021503,-0.013019,-0.004494,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,-0.00579,-0.009054,-0.016005,-0.02212,-0.026846,-0.031845,-0.043748,-0.047283,-0.053139,-0.056623,-0.060338,-0.056295,-0.051367,-0.042632,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004418,-0.005755,-0.004083,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004083,-0.00471,-0.008799,-0.011591,-0.014759,-0.019285,-0.024672,-0.029071,...,-0.135433,-0.11648,-0.093951,-0.072577,-0.052364,-0.036383,-0.021503,-0.013019,-0.004494,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,-0.00579,-0.009054,-0.016005,-0.02212,-0.026846,-0.031845,-0.043748,-0.047283,-0.053139,-0.056623,-0.060338,-0.056295,-0.051367,-0.042632,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004418,-0.005755,-0.004083,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004083,-0.00471,-0.008799,-0.011591,-0.014759,-0.019285,-0.024672,-0.029071,...,-0.135433,-0.11648,-0.093951,-0.072577,-0.052364,-0.036383,-0.021503,-0.013019,-0.004494,-0.004083,0.0,0.0,0.0,0.0,0.0,0.0,-0.00579,-0.009054,-0.016005,-0.02212,-0.026846,-0.031845,-0.043748,-0.047283,-0.053139,-0.056623,-0.060338,-0.056295,-0.051367,-0.042632,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0


In [90]:
y_train.head(3)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [91]:
def create_layer(n_neurons, n_inputs):
    weights = np.zeros((n_neurons, n_inputs + 1)) 
    for i in range(n_neurons):
        for j in range(1, n_inputs + 1):
            weights[i][j] = np.random.uniform(-0.1, 0.1)
    return weights

In [92]:
hidden_layer_w = create_layer(25, 784) # Neurons and their weights
hidden_layer_y = np.zeros(25) # Array for saving outputs
hidden_layer_error = np.zeros(25) # Array for saving error values (used in backpropagation)

output_layer_w = create_layer(10, 25)
output_layer_y = np.zeros(10)
output_layer_error = np.zeros(10)

In [93]:
def print_train_progress(n_epoch, n_train_correct, n_test_correct, x_train_l, x_test_l):
    print(f"""Epoch number: {n_epoch}\n Train accuracy: {n_train_correct / x_train_l}\n Test accuracy: {n_test_correct / x_test_l}""")

In [94]:
def forward_pass(hidden_layer_y, output_layer_y, x): # Calculate activation function outputs for each layer and store them
    # Activation function for hidden layer
    for ind, weight in enumerate(hidden_layer_w):
        z = np.dot(weight, x) # Summed weights
        hidden_layer_y[ind] = np.tanh(z) # Tanh (-1 to 1)
    hidden_output_arr = np.concatenate((np.array([1.0]), hidden_layer_y))
    # Activation function for output layer
    for ind, weight in enumerate(output_layer_w):
        z = np.dot(weight, hidden_output_arr)
        output_layer_y[i] = 1.0 / (1.0 + np.exp(-z)) # Sigmoid (0 to 1), only for output layer
    return hidden_layer_y, output_layer_y

In [95]:
def backward_pass(hidden_layer_error, output_layer_error, y_truth): # Backpropagate the output layer error
    # Backpropagate error for each output neuron
    # and create array of all output neuron errors.
    for ind, y in enumerate(output_layer_y):
        error_prime = -(y_truth[ind] - y) # Loss derivative
        derivative = y * (1.0 - y) # Logistic derivative
        output_layer_error[i] = error_prime * derivative
    for ind, y in enumerate(hidden_layer_y):
        # Create array weights connecting the output of
        # hidden neuron i to neurons in the output layer.
        error_weights = []
        for weight in output_layer_w:
            error_weights.append(weight[ind + 1])
        error_weight_array = np.array(error_weights)
        # Backpropagate error for hidden neuron.
        derivative = 1.0 - y**2 # tanh derivative
        weighted_error = np.dot(error_weight_array, output_layer_error)
        hidden_layer_error[ind] = weighted_error * derivative
    return hidden_layer_error, output_layer_error

In [96]:
def adjust_weights(output_layer_w, hidden_layer_w, x, lr):
    for ind, error in enumerate(hidden_layer_error):
        hidden_layer_w[ind] -= (x * lr * error) # Update all weights
    hidden_output_array = np.concatenate((np.array([1.0]), hidden_layer_y))
    for ind, error in enumerate(output_layer_error):
        output_layer_w[ind] -= (hidden_output_array * lr * error)
    return output_layer_w, hidden_layer_w

In [97]:
LEARNING_RATE = 0.1
EPOCHS = 20
index_list = np.array(range(len(X_train)))
for i in range(1, EPOCHS + 1):
    np.random.shuffle(index_list)
    train_res_correct = 0
    for j in index_list: # Training loop, complete forward- and backward passes, and then adjust the weights
        x_with_bias = np.concatenate((np.array([1.0]), X_train.iloc[j])) # Add the bias term to the start of neuron weights
        hidden_layer_y, output_layer_y = forward_pass(hidden_layer_y, output_layer_y, x_with_bias)
        if np.argmax(output_layer_y) == np.argmax(y_train.iloc[j]):
            train_res_correct += 1
        hidden_layer_error, output_layer_error = backward_pass(hidden_layer_error, output_layer_error, y_train.iloc[j])
        output_layer_w, hidden_layer_w = adjust_weights(output_layer_w, hidden_layer_w, x_with_bias, LEARNING_RATE)

    test_res_correct = 0
    for j in range(len(X_test)): # Evaluate network
        x_with_bias = np.concatenate((np.array([1.0]), X_test.iloc[j]))
        hidden_layer_y, output_layer_y = forward_pass(hidden_layer_y, output_layer_y, x_with_bias)
        if np.argmax(output_layer_y) == np.argmax(y_test.iloc[j]):
            test_res_correct += 1

    print_train_progress(i, train_res_correct, test_res_correct, len(X_train), len(X_test))

Epoch number: 1
 Train accuracy: 0.11236666666666667
 Test accuracy: 0.1135
Epoch number: 2
 Train accuracy: 0.12531666666666666
 Test accuracy: 0.127
Epoch number: 3
 Train accuracy: 0.10796666666666667
 Test accuracy: 0.1095
Epoch number: 4
 Train accuracy: 0.09776666666666667
 Test accuracy: 0.1017
Epoch number: 5
 Train accuracy: 0.10851666666666666
 Test accuracy: 0.1104
Epoch number: 6
 Train accuracy: 0.10221666666666666
 Test accuracy: 0.1026


KeyboardInterrupt: ignored